<a href="https://colab.research.google.com/github/sumanthd17/aspect-based-sentiment/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=e16cdfb057d725865d4384c5f5a63a55284969b4d514328ca2ac53d47229f058
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 160.4 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.3MB/s 
     |████████████████████████████████| 890kB 13.8MB/s 
     |████████████████████████████████| 3.0MB 23.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ef5e53386676d1419cac7cf0e6ec96f9a618db29689fe0f6180afb25f35b8d6b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import pandas as pd
from tqdm import tqdm

import torch
import transformers as optimus

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!git clone https://github.com/sumanthd17/aspect-based-sentiment.git

Cloning into 'aspect-based-sentiment'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 5), reused 17 (delta 3), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [5]:
cd aspect-based-sentiment

/content/aspect-based-sentiment


In [6]:
def load_train_data(input_dir):
    df = pd.read_csv(input_dir + "train-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [7]:
def hyper_params():
    BATCH_SIZE = 32
    MAX_SEQ_LENGTH = 128
    LEARNING_RATE = 2e-5
    EPOCHS = 5
    return BATCH_SIZE, MAX_SEQ_LENGTH, LEARNING_RATE, EPOCHS

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
!python create_data.py

2977
(1430, ['LOCATION1', 'is', 'transforming', 'and', 'the', 'prices', 'will', 'go', 'up', 'and', 'up'], 'LOCATION1', 'price', 'Negative')
45025


In [10]:
train_data = load_train_data('QA_pairs/')

batch_size, max_seq_len, lr, epochs = hyper_params()
num_training_steps = int(len(train_data) / batch_size)
print(len(train_data))

45025


In [11]:
tokenizer_class, pretrained_weights = (
    optimus.BertTokenizer,
    "bert-base-uncased",
)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [12]:
train_sample = train_data[:10000]

In [13]:
tokenized = pd.DataFrame()

In [14]:
question_tokens = train_sample['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
q_segment_tokens = question_tokens.apply(lambda x: [0]*len(x))

In [15]:
answer_tokens = train_sample['ans'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
answer_tokens = answer_tokens.apply(lambda x: [i for i in x if i != 101])
a_segment_tokens = answer_tokens.apply(lambda x: [1]*len(x))

In [16]:
tokenized['input_ids'] = question_tokens + answer_tokens

In [17]:
tokenized['segment_ids'] = q_segment_tokens + a_segment_tokens

In [18]:
tokenized['input_mask'] = ''
for i, row in tokenized.iterrows():
  tokenized.iloc[i]['input_mask'] = [1]*len(tokenized.iloc[i]['input_ids'])

In [19]:
tokenized['input_mask'] = tokenized['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [20]:
tokenized['input_ids'] = tokenized['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [21]:
tokenized['segment_ids'] = tokenized['segment_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [22]:
sent2idx = {
    'None': 0,
    'Positive': 1,
    'Negative': 2
}

In [23]:
tokenized['label'] = ''
tokenized['label'] = train_sample['sentiment'].apply(lambda x: sent2idx[x])

In [24]:
train_data['sentiment'].value_counts()

None        41629
Positive     2475
Negative      921
Name: sentiment, dtype: int64

In [25]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [26]:
input_ids = torch.tensor([r for r in tokenized['input_ids']], dtype=torch.long)
input_masks = torch.tensor([r for r in tokenized['input_mask']], dtype=torch.long)
label_ids = torch.tensor([r for r in tokenized['label']], dtype=torch.long)
segment_ids = torch.tensor([r for r in tokenized['segment_ids']], dtype=torch.long)

In [27]:
train_dataset = TensorDataset(input_ids, input_masks, segment_ids, label_ids)

In [28]:
train_sampler = RandomSampler(train_dataset)

In [29]:
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [30]:
model_class, pretrained_weights = (
    optimus.BertForSequenceClassification,
    "bert-base-uncased",
)

model = model_class.from_pretrained(pretrained_weights, num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [31]:
model = model.to(device)

In [32]:
optimizer_params = [
  {'params': [p for n, p in model.named_parameters()], 'weight_decay_rate': 0.0}
]

In [33]:
optimizer = optimus.AdamW(model.parameters(), lr=lr, correct_bias=False)
scheduler = optimus.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

In [42]:
model.train()
for step, batch in enumerate(tqdm(train_dataloader)):
  input_ids, input_mask, segment_ids, label_ids = batch
  
  input_ids = input_ids.to(device)
  input_mask = input_mask.to(device)
  segment_ids = segment_ids.to(device)
  label_ids = label_ids.to(device)

  loss, _ = model(input_ids=input_ids, attention_mask=input_mask, token_type_ids=segment_ids, labels=label_ids)

  loss.backward()
  optimizer.step()
  scheduler.step()



  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 1/313 [00:01<06:45,  1.30s/it]

  1%|          | 2/313 [00:02<06:32,  1.26s/it]

  1%|          | 3/313 [00:03<06:23,  1.24s/it]

  1%|▏         | 4/313 [00:04<06:16,  1.22s/it]

  2%|▏         | 5/313 [00:05<06:10,  1.20s/it]

  2%|▏         | 6/313 [00:07<06:08,  1.20s/it]

  2%|▏         | 7/313 [00:08<06:03,  1.19s/it]

  3%|▎         | 8/313 [00:09<06:02,  1.19s/it]

  3%|▎         | 9/313 [00:10<05:59,  1.18s/it]

  3%|▎         | 10/313 [00:11<05:57,  1.18s/it]

  4%|▎         | 11/313 [00:13<05:56,  1.18s/it]

  4%|▍         | 12/313 [00:14<05:54,  1.18s/it]

  4%|▍         | 13/313 [00:15<05:53,  1.18s/it]

  4%|▍         | 14/313 [00:16<05:51,  1.18s/it]

  5%|▍         | 15/313 [00:17<05:51,  1.18s/it]

  5%|▌         | 16/313 [00:18<05:50,  1.18s/it]

  5%|▌         | 17/313 [00:20<05:49,  1.18s/it]

  6%|▌         | 18/313 [00:21<05:47,  1.18s/it]

  6%|▌         | 19/313 [00:22<05:47,  1.18s/it]

  6%|▋         |